In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Exploratory Data Analysis\n",
    "## Football Alpha Analysis - 2025-26 Season\n",
    "\n",
    "This notebook explores the dataset containing player statistics from Europe's Top 5 leagues."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import sys\n",
    "sys.path.append('../src')\n",
    "from analysis import get_data\n",
    "\n",
    "plt.style.use('seaborn-v0_8-whitegrid')\n",
    "pd.set_option('display.max_columns', 50)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data from AWS Athena"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = get_data()\n",
    "print(f\"Dataset Shape: {df.shape}\")\n",
    "print(f\"Total Players: {len(df)}\")\n",
    "print(f\"Total Columns: {len(df.columns)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Overview"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df.head(10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df.describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Missing Values Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "missing = df.isnull().sum()\n",
    "missing_pct = (missing / len(df) * 100).round(2)\n",
    "missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})\n",
    "missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize missing values\n",
    "plt.figure(figsize=(12, 6))\n",
    "missing_cols = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False).head(20)\n",
    "plt.barh(missing_cols.index, missing_cols['Percentage'])\n",
    "plt.xlabel('Missing %')\n",
    "plt.title('Top 20 Columns with Missing Values')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. League Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "league_counts = df['comp'].value_counts()\n",
    "print(league_counts)\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.pie(league_counts.values, labels=league_counts.index, autopct='%1.1f%%', startangle=90)\n",
    "plt.title('Player Distribution by League')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Position Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_main_position(pos):\n",
    "    if pd.isna(pos):\n",
    "        return 'Unknown'\n",
    "    pos = pos.upper()\n",
    "    if 'GK' in pos:\n",
    "        return 'GK'\n",
    "    elif 'DF' in pos:\n",
    "        return 'DF'\n",
    "    elif 'MF' in pos:\n",
    "        return 'MF'\n",
    "    elif 'FW' in pos:\n",
    "        return 'FW'\n",
    "    return 'Unknown'\n",
    "\n",
    "df['main_pos'] = df['pos'].apply(get_main_position)\n",
    "pos_counts = df['main_pos'].value_counts()\n",
    "\n",
    "plt.figure(figsize=(8, 6))\n",
    "colors = {'GK': '#9b59b6', 'DF': '#3498db', 'MF': '#2ecc71', 'FW': '#e74c3c'}\n",
    "plt.bar(pos_counts.index, pos_counts.values, color=[colors.get(p, '#95a5a6') for p in pos_counts.index])\n",
    "plt.xlabel('Position')\n",
    "plt.ylabel('Count')\n",
    "plt.title('Player Distribution by Position')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Age Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 5))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.hist(df['age'], bins=20, edgecolor='black', color='#3498db')\n",
    "plt.xlabel('Age')\n",
    "plt.ylabel('Count')\n",
    "plt.title('Age Distribution')\n",
    "plt.axvline(df['age'].mean(), color='red', linestyle='--', label=f\"Mean: {df['age'].mean():.1f}\")\n",
    "plt.legend()\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "df.boxplot(column='age', by='main_pos', ax=plt.gca())\n",
    "plt.suptitle('')\n",
    "plt.title('Age by Position')\n",
    "plt.xlabel('Position')\n",
    "plt.ylabel('Age')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Goals and Assists Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n",
    "\n",
    "# Goals histogram\n",
    "axes[0, 0].hist(df['gls'], bins=30, edgecolor='black', color='#3498db')\n",
    "axes[0, 0].set_xlabel('Goals')\n",
    "axes[0, 0].set_ylabel('Count')\n",
    "axes[0, 0].set_title('Goals Distribution')\n",
    "\n",
    "# Assists histogram\n",
    "axes[0, 1].hist(df['ast'], bins=30, edgecolor='black', color='#2ecc71')\n",
    "axes[0, 1].set_xlabel('Assists')\n",
    "axes[0, 1].set_ylabel('Count')\n",
    "axes[0, 1].set_title('Assists Distribution')\n",
    "\n",
    "# Goals by position\n",
    "df.boxplot(column='gls', by='main_pos', ax=axes[1, 0])\n",
    "axes[1, 0].set_title('Goals by Position')\n",
    "axes[1, 0].set_xlabel('Position')\n",
    "axes[1, 0].set_ylabel('Goals')\n",
    "\n",
    "# Assists by position\n",
    "df.boxplot(column='ast', by='main_pos', ax=axes[1, 1])\n",
    "axes[1, 1].set_title('Assists by Position')\n",
    "axes[1, 1].set_xlabel('Position')\n",
    "axes[1, 1].set_ylabel('Assists')\n",
    "\n",
    "plt.suptitle('')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Key metrics correlation\n",
    "key_metrics = ['gls', 'ast', 'xg', 'xag', 'finishing_alpha', 'playmaking_alpha', 'col_90s', 'age']\n",
    "corr_matrix = df[key_metrics].corr()\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0, fmt='.2f')\n",
    "plt.title('Correlation Matrix - Key Metrics')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Top Players Overview"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Top 10 Goal Scorers:\")\n",
    "df.nlargest(10, 'gls')[['player', 'squad', 'comp', 'gls', 'xg', 'finishing_alpha']]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Top 10 Assist Providers:\")\n",
    "df.nlargest(10, 'ast')[['player', 'squad', 'comp', 'ast', 'xag', 'playmaking_alpha']]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Summary Statistics by League"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "league_summary = df.groupby('comp').agg({\n",
    "    'player': 'count',\n",
    "    'gls': ['sum', 'mean'],\n",
    "    'ast': ['sum', 'mean'],\n",
    "    'xg': ['sum', 'mean'],\n",
    "    'finishing_alpha': 'mean',\n",
    "    'playmaking_alpha': 'mean',\n",
    "    'age': 'mean'\n",
    "}).round(2)\n",
    "\n",
    "league_summary.columns = ['Players', 'Total Goals', 'Avg Goals', 'Total Assists', 'Avg Assists', \n",
    "                          'Total xG', 'Avg xG', 'Avg Finishing Alpha', 'Avg Playmaking Alpha', 'Avg Age']\n",
    "league_summary"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Key Findings\n",
    "\n",
    "1. **Dataset Size:** 1,369 players with 5+ 90s played\n",
    "2. **Leagues:** Premier League, La Liga, Serie A, Bundesliga, Ligue 1\n",
    "3. **Position Distribution:** Midfielders most common, goalkeepers least\n",
    "4. **Age:** Average age around 26-27 years\n",
    "5. **Goals:** Heavily right-skewed distribution (most players score few goals)\n",
    "6. **Correlation:** Strong correlation between xG and actual goals"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}